In [1]:
import os
from basic_rag_utils.helper import get_openai_api_key, get_gemini_api_key
# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
OPENAI_API_KEY = get_openai_api_key()
GEMINI_API_KEY = get_gemini_api_key()

In [ ]:
# False: Generate the embedding for the dataset. (Associated cost with using OpenAI endpoint)
# True: Load the dataset that already has the embedding vectors.
load_embedding = False